In [1]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# 近代数学とプログラミング

## 関数論と四則演算

* (1903 &ndash; 1995) [Alonzo Church - Wikipedia](https://en.wikipedia.org/wiki/Alonzo_Church)
* [Church encoding - Wikipedia](https://en.wikipedia.org/wiki/Church_encoding)
  - [Adding church numerals using lambda functions in python - Stack Overflow](https://stackoverflow.com/questions/24905241/adding-church-numerals-using-lambda-functions-in-python)

In [3]:
def f(x):
    return x+1

In [4]:
0, f(0), f(f(0)), f(f(f(0)))

(0, 1, 2, 3)

In [5]:
f = lambda x: x+1

In [6]:
0, f(0), f(f(0)), f(f(f(0)))

(0, 1, 2, 3)

### 関数の合成回数で序数を定義する
* [Function composition - Wikipedia](https://en.wikipedia.org/wiki/Function_composition)

In [7]:
def zero(f):
    return lambda x: x

In [8]:
def one(f):
    return lambda x: f(x)

In [9]:
def two(f):
    return lambda x: f(f(x))

In [10]:
def three(f):
    return lambda x: f(f(f(x)))

In [11]:
f

<function __main__.<lambda>(x)>

In [12]:
zero

<function __main__.zero(f)>

In [13]:
zero(f)

<function __main__.zero.<locals>.<lambda>(x)>

In [14]:
(zero(f))(0)

0

In [15]:
(one(f))(0)

1

In [16]:
(two(f))(0)

2

In [17]:
(three(f))(0)

3

### 関数の加算

In [18]:
def church_add(m, n):
    return lambda f: lambda x: m(f)(n(f)(x))

In [19]:
church_add(one,two)

<function __main__.church_add.<locals>.<lambda>(f)>

In [20]:
church_add(one,two)(f)

<function __main__.church_add.<locals>.<lambda>.<locals>.<lambda>(x)>

In [21]:
(church_add(one,two)(f))(0)

3

In [22]:
(church_add(three,two)(f))(0)

5

### 関数の乗算

In [70]:
def church_mul(m, n):
    return lambda f: lambda x: m(n(f))(x)

In [71]:
church_mul(one, two)(f)(0)

2

In [72]:
church_mul(two, two)(f)(0)

4

In [73]:
church_mul(two, three)(f)(0)

6

### 後者関数

In [74]:
def church_succ(n):
    return lambda f: lambda x: f(n(f)(x))

In [75]:
def church_zero(f):
    return lambda x: x

In [76]:
def church_one(f):
    return lambda x: (church_succ(church_zero))(f)(x)

In [77]:
def church_two(f):
    return lambda x: (church_succ(church_succ(church_zero)))(f)(x)

In [78]:
church_zero(f)(0)

0

In [79]:
(church_succ(church_zero))(f)(0)

1

In [80]:
church_one(f)(0)

1

In [81]:
church_two(f)(0)

2

課題-7) 次のように定義する ``nest_lit()`` を使って、２つの整数の足し算をする関数 ``nest_add()`` を定義せよ。

In [27]:
def nest_list(f=lambda x: x+1, x=0):
    while True:
        yield x
        x = f(x)

def nest_add(a, b):
    ### BEGIN SOLUTION
    g = nest_list()
    {next(g) for _ in range(a)}
    {next(g) for _ in range(b)}
    return next(g)
    ### END SOLUTION

In [28]:
nest_add(5,3)

8

---
以下、採点用のセルにつき編集できない:

In [29]:
from nose.tools import assert_equal, assert_true

In [ ]:
assert_equal(nest_add(5, 3), 8)